## 移除最后conv最后一层网络

使用兩個fc


In [3]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [10]:
def valid_class_acc(classify_model, valid_data_loader):
    classify_model = classify_model.eval()
    predict = []
    total = len(valid_data_loader.dataset)
    for idx in range(total):
        embed, _ = valid_data_loader.dataset[idx]
        p = classify_model(embed).argmax().item()
        predict.append(p)
        print('\r %d / %d' % (idx, total), end='')
        sys.stdout.flush()
    
    df_refer = valid_data_loader.dataset.refer
    df_refer["predict"] = predict
    df_refer['correct'] = df_refer.predict == df_refer.disease_class
    accuracy = (df_refer.correct == True).sum()/len(df_refer)
    return accuracy*100

### set high parameters

In [5]:
batch_size = 256
num_epochs = 300
extract_size = 1024
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_conv.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_remove_last_conv.h')

# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm1layer.h")

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [8]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = MultiClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm1layer_last.pkl'))

### time to train model

In [9]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [11]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(1, num_epochs+1):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('')        
    valid_acc = valid_class_acc(classify_model, valid_data_loader)
    print('\n Epoch {}, spent time:{:.2f}s, valid: {:.2f}%'.format(epoch, time.time()-start, valid_acc))       
    if epoch%1 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multify_rm1layer_%d.pkl' % epoch))
# torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multify_rm1layer_last.pkl'))

Epoch [1/300], Step [120/127], Loss: 1.5641, Accuracy: 54.69%, Best_acc: 56.25%            
 4981 / 4982
 Epoch 1, spent time:14.55s, valid: 55.16%
Epoch [2/300], Step [120/127], Loss: 1.1984, Accuracy: 68.75%, Best_acc: 68.75%            
 4981 / 4982
 Epoch 2, spent time:14.45s, valid: 58.13%
Epoch [3/300], Step [120/127], Loss: 1.0833, Accuracy: 65.23%, Best_acc: 68.75%            
 4981 / 4982
 Epoch 3, spent time:14.38s, valid: 61.90%
Epoch [4/300], Step [120/127], Loss: 0.9942, Accuracy: 67.58%, Best_acc: 73.44%            
 4981 / 4982
 Epoch 4, spent time:14.39s, valid: 64.73%
Epoch [5/300], Step [120/127], Loss: 0.9118, Accuracy: 69.92%, Best_acc: 75.00%            
 4981 / 4982
 Epoch 5, spent time:14.62s, valid: 66.72%
Epoch [6/300], Step [120/127], Loss: 0.8006, Accuracy: 73.83%, Best_acc: 75.39%            
 4981 / 4982
 Epoch 6, spent time:14.31s, valid: 67.24%
Epoch [7/300], Step [120/127], Loss: 0.8505, Accuracy: 73.83%, Best_acc: 75.39%            
 4981 / 4982
 Epoch 

Epoch [56/300], Step [120/127], Loss: 0.5099, Accuracy: 78.91%, Best_acc: 88.67%            
 4981 / 4982
 Epoch 56, spent time:14.22s, valid: 75.81%
Epoch [57/300], Step [120/127], Loss: 0.3424, Accuracy: 85.55%, Best_acc: 88.67%            
 4981 / 4982
 Epoch 57, spent time:14.32s, valid: 76.13%
Epoch [58/300], Step [120/127], Loss: 0.2973, Accuracy: 88.67%, Best_acc: 88.67%            
 4981 / 4982
 Epoch 58, spent time:14.41s, valid: 77.12%
Epoch [59/300], Step [120/127], Loss: 0.2863, Accuracy: 87.11%, Best_acc: 88.67%            
 4981 / 4982
 Epoch 59, spent time:14.39s, valid: 76.78%
Epoch [60/300], Step [120/127], Loss: 0.3693, Accuracy: 82.81%, Best_acc: 88.67%            
 4981 / 4982
 Epoch 60, spent time:14.34s, valid: 77.86%
Epoch [61/300], Step [120/127], Loss: 0.3409, Accuracy: 83.20%, Best_acc: 88.67%            
 4981 / 4982
 Epoch 61, spent time:14.35s, valid: 76.52%
Epoch [62/300], Step [120/127], Loss: 0.4080, Accuracy: 81.25%, Best_acc: 89.06%            
 4981 /

KeyboardInterrupt: 

### How good is the model?

In [6]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm1layer.h")

In [21]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm1layer_100.pkl'))

In [22]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 77.56%.


## With 100 epoch the result is the best,yet.

In [23]:
!mv ./models/class_multify_rm1layer_100.pkl ./models/class_multify_rm1layer_good.pkl